## Demo of buoyancy driven stokes flow in an Annulus

In [ ]:
import underworld as uw
import glucifer
import numpy as np
from underworld import function as fn

In [ ]:
annulus = uw.mesh._FeMesh_Annulus(elementRes=(20,120), 
                                  radialLengths=(2.0,4.), angularExtent=(0.0,360.0),
                                  periodic = [False, True])
tField = uw.mesh.MeshVariable(annulus, nodeDofCount=1)
vField = uw.mesh.MeshVariable(annulus, nodeDofCount=2)
pField = uw.mesh.MeshVariable(annulus.subMesh, nodeDofCount=1)

In [ ]:
fig = glucifer.Figure()
fig.append(glucifer.objects.Mesh(annulus, segmentsPerEdge=1))
fig.append(glucifer.objects.Surface(annulus, tField, onMesh=True ))
fig.show()

In [ ]:
t_outer = 5.0
t_inner = 10.0
outer = annulus.specialSets["MaxI_VertexSet"]
inner = annulus.specialSets["MinI_VertexSet"]

tField.data[inner.data] = 10.0
tField.data[outer.data] = 5.0

# setup parameters for temperature distribution
dr = annulus.radialLengths[1] - annulus.radialLengths[0]
dT_dr = (t_outer-t_inner)/(dr)
c0 = t_inner - dT_dr*annulus.radialLengths[0]
# print c0, dT_dr

# wavenumber for perturbation
k = 4.

for ind,coord in enumerate(annulus.data):
    r = np.sqrt(coord[0]**2 + coord[1]**2)
    theta = np.arctan2(coord[1], coord[0])
    
    pert = np.sin((r-2.)*np.pi/dr)*np.cos(k*theta)
    
    t = r*dT_dr + c0
    tField.data[ind] = t + 1.*pert

In [ ]:
fig.show()

In [ ]:
vBC = uw.conditions.RotatedDirichletCondition( variable=vField, indexSetsPerDof=(inner+outer, None),
                                             basis_vectors=(annulus.rot_vec_normal,annulus.rot_vec_tangent))
tBC = uw.conditions.DirichletCondition( variable=tField, indexSetsPerDof=(inner+outer))

In [ ]:
# tDotField = uw.mesh.MeshVariable(annulus, nodeDofCount=1)
# advDiffSLE = uw.systems.AdvectionDiffusion(tField, tDotField, vField, fn_diffusivity=1.0, conditions=tBC)

In [ ]:
z_hat  = -1.0*annulus.fn_unitvec_radial()
Ra = 1e4

In [ ]:
bodyForceFn = Ra * -1.*tField * z_hat

In [ ]:
stokesSLE = uw.systems.Stokes( vField, pField, 
                               fn_viscosity=1.0, fn_bodyforce=bodyForceFn, 
                               conditions=vBC, _removeBCs=False)

In [ ]:
stokesSolver = uw.systems.Solver(stokesSLE)

In [ ]:
stokesSolver.solve()
uw.libUnderworld.Underworld.AXequalsX( stokesSLE._rot._cself, stokesSLE._velocitySol._cself, False)

In [ ]:
# setup analysis function
vdotv = fn.math.dot(vField,vField)
v2sum_integral  = uw.utils.Integral( mesh=annulus, fn=vdotv )
volume_integral = uw.utils.Integral( mesh=annulus, fn=1. )
velmag = fn.math.sqrt(vdotv)
forcemag = fn.math.sqrt(fn.math.dot(bodyForceFn,bodyForceFn))

In [ ]:
figV = glucifer.Figure(**fig)
figV.append(glucifer.objects.Mesh(annulus, segmentsPerEdge=1))
figV.append(glucifer.objects.Surface(annulus, tField, onMesh=True))
figV.append(glucifer.objects.VectorArrows(annulus, vField/1e3))
figV.show()

In [ ]:
# create checkpoint function
def checkpoint( mesh, fieldDict, swarm, swarmDict, index,
                meshName='mesh', swarmName='swarm', 
                prefix='./', enable_xdmf=True):
    import os
    # Check the prefix is valid
    if prefix is not None:
        if not prefix.endswith('/'): prefix += '/' # add a backslash
        if not os.path.exists(prefix) and uw.rank()==0:
            print "Creating directory: ",prefix 
            os.makedirs(prefix)
        uw.barrier() 
            
    if not isinstance(index, int):
        raise TypeError("'index' is not of type int")        
    ii = str(index)
    
    if mesh is not None:
        
        # Error check the mesh and fields
        if not isinstance(mesh, uw.mesh.FeMesh):
            raise TypeError("'mesh' is not of type uw.mesh.FeMesh")
        if not isinstance(fieldDict, dict):
            raise TypeError("'fieldDict' is not of type dict")
        for key, value in fieldDict.iteritems():
            if not isinstance( value, uw.mesh.MeshVariable ):
                raise TypeError("'fieldDict' must contain uw.mesh.MeshVariable elements")


        # see if we have already saved the mesh. It only needs to be saved once
        if not hasattr( checkpoint, 'mH' ):
            checkpoint.mH = mesh.save(prefix+meshName+".h5")
        mh = checkpoint.mH

        for key,value in fieldDict.iteritems():
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, mh, meshName)
        
    # is there a swarm
    if swarm is not None:
        
        # Error check the swarms
        if not isinstance(swarm, uw.swarm.Swarm):
            raise TypeError("'swarm' is not of type uw.swarm.Swarm")
        if not isinstance(swarmDict, dict):
            raise TypeError("'swarmDict' is not of type dict")
        for key, value in swarmDict.iteritems():
            if not isinstance( value, uw.swarm.SwarmVariable ):
                raise TypeError("'fieldDict' must contain uw.swarm.SwarmVariable elements")
    
        sH = swarm.save(prefix+swarmName+"-"+ii+".h5")
        for key,value in swarmDict.iteritems():
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, sH, swarmName)


In [ ]:
# xdmf output
fieldDict = {'velocity':vField,
             'temperature':tField}
checkpoint(annulus, fieldDict, None, None, index=0, prefix='output')